In [1]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\feder\Documents\GitHub\relgan\src\jupyter_tests


In [2]:
os.chdir("..")
cwd = os.getcwd()
print(cwd)
assert cwd.endswith("src")

C:\Users\feder\Documents\GitHub\relgan\src


# CR dataset management

In [28]:
import nltk, pandas as pd, numpy as np
from os.path import join
from tqdm import tqdm
from utils.text_process import get_word_list, get_dict, text_to_code
from utils.static_file_manage import write_json

In [48]:
file = [
    'Apex AD2600 Progressive-scan DVD player.txt',
    'Canon G3.txt',
    'Creative Labs Nomad Jukebox Zen Xtra 40GB.txt',
    'Nokia 6610.txt',
    'Nikon coolpix 4300.txt',
]
data_dir = join(os.getcwd(), "..", "resources", "data", "MovieReviews", "cr")

In [7]:
def produce_line(_line):
    _line = _line.strip()
    _line = _line.lower()
    _token = nltk.word_tokenize(_line)
    return _token


def pro_file(file, pos_list, neg_list):
    with open(file, 'r') as f:
        for item in f:
            item = item.strip()
            f_p = 0
            f_n = 0
            f_p = item.find('[+')
            f_n = item.find('[-')
            if f_p != -1:
                pos = item.find('##') + 2
                line_text = item[pos:]
                line_list = produce_line(line_text)
                l = len(line_list)
                if l <= 1500000000 and l > 1:
                    pos_list.append(line_list)
            # print('===============')
            # print(line_text)
            # print(item)
            # input()
            if f_n != -1:
                pos = item.find('##') + 2
                line_text = item[pos:]
                line_list = produce_line(line_text)
                l = len(line_list)
                if l <= 1500000000000 and l > 1:
                    neg_list.append(line_list)
            # print('===============')
            # print(line_text)
            # print(item)
            # input()
    return pos_list, neg_list

In [13]:
pos_list = []
neg_list = []

for file_i in file:
    pos_list, neg_list = pro_file(os.path.join(data_dir, "Reviews-9-products", file_i), pos_list, neg_list)

In [14]:
df = pd.DataFrame(list(zip(np.concatenate([np.ones(len(pos_list), dtype=int), np.zeros(len(neg_list), dtype=int)]), pos_list + neg_list)), columns=["sentiment", "text"])

In [19]:
df['len'] = df['text'].apply(len)
df = df[df['len'] <= 15]

In [21]:
token_list = [x for sent in df['text'] for x in sent]

In [23]:
from collections import Counter 
def removeElements(lst, k): 
    counted = Counter(lst) 
    return [el for el in lst if counted[el] >= k] 

token_list_removed = removeElements(token_list, 10) #at least 10 times

In [38]:
word_set = list(dict.fromkeys(set(token_list)))
[word_index_dict, index_word_dict] = get_dict(word_set)
sequence_len = len(max(token_list, key=len))

In [44]:
def apply_text_to_code(x):
    tokenization = nltk.word_tokenize(x.lower())
    eof_code = len(word_index_dict)  # used to filled in the blank to make up a sentence with seq_len
    index = 0
    code_int = []
    for word in tokenization:
        try:
            code_int.append(int(word_index_dict[word]))
        except KeyError:
            return np.nan
        index += 1
    while index < sequence_len:
        code_int.append(int(eof_code))
        index += 1
    return code_int

In [45]:
df['text'] = df['text'].apply(lambda x: " ".join(x))
df['tokenized_text'] = df['text'].apply(lambda x: apply_text_to_code(x))
df.dropna(inplace=True)

C:\Users\feder\Anaconda3\envs\RelGAN\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\feder\Anaconda3\envs\RelGAN\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\feder\Anaconda3\envs\RelGAN\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [51]:
df.to_csv(join(data_dir, 'train' + '.csv'))

In [54]:
config_dict = {}
config_dict['seq_len'] = sequence_len
config_dict['vocabulary_size'] = len(word_index_dict) + 1
config_dict['sentiment_num'] = max(df['sentiment']) + 1

write_json(join(data_dir, 'word_index_dict.json'), word_index_dict)
write_json(join(data_dir, 'index_word_dict.json'), index_word_dict)
write_json(join(data_dir, 'config.json'), config_dict)

In [52]:
df

,sentiment,text,len,tokenized_text
0,1,im a more happier person after discovering the...,11,"[261, 971, 163, 1000, 241, 320, 120, 1274, 255..."
2,1,it practically plays almost everything you giv...,9,"[968, 399, 740, 46, 1049, 354, 767, 968, 712, ..."
4,1,for the price it is a well spent investment !,10,"[350, 1274, 175, 968, 745, 971, 1038, 1010, 11..."
5,1,"this is by far the nicest one , in so many ways .",13,"[436, 745, 1349, 516, 1274, 390, 253, 114, 678..."
8,1,"it plays alternate video formats ( vcds , svcd...",15,"[968, 740, 1165, 1098, 152, 542, 792, 114, 120..."
...,...,...,...,...
1745,0,one downside though is that autofocus does not...,15,"[253, 945, 254, 745, 499, 1197, 594, 234, 349,..."
1748,0,"it has a 3x optical zoom , which is average fo...",14,"[968, 938, 971, 1117, 699, 1219, 114, 10, 745,..."
1753,0,"i only have one complaint , and that is the 8m...",14,"[858, 479, 171, 253, 1079, 114, 1133, 499, 745..."
1754,0,8mb for a camera like this is a joke .,10,"[1203, 350, 971, 136, 826, 436, 745, 971, 1040..."
